# Beyond The Crosshair

In [212]:
import pandas as pd

## Clean Data Loading

In [ ]:
cs16_data = pd.read_csv("../assets/clean/SteamDB_Counter-Strike_1.6_Clean.csv", parse_dates=["month"])
css_data = pd.read_csv("../assets/clean/SteamDB_Counter-Strike_Source_Clean.csv", parse_dates=["month"])
cscz_data = pd.read_csv("../assets/clean/SteamDB_Counter-Strike_Condition_Zero_Clean.csv", parse_dates=["month"])
csgo_data = pd.read_csv("../assets/clean/SteamDB_Counter-Strike_Global_Offensive_Clean.csv", parse_dates=["month"])
cs2_data = pd.read_csv("../assets/clean/SteamDB_Counter-Strike_2_Clean.csv", parse_dates=["month"])
cs_merged_data = pd.read_csv("../assets/clean/SteamDB_Counter-Strike_Clean.csv", parse_dates=["month"])

v_data = pd.read_csv("../assets/clean/GG_Valorant_Clean.csv", parse_dates=["month"])
fn_data = pd.read_csv("../assets/clean/GG_Fortnite_Clean.csv", parse_dates=["month"])
rss_data = pd.read_csv("../assets/clean/GG_Rainbow_Six_Siege_Clean.csv", parse_dates=["month"])
cod_data = pd.read_csv("../assets/clean/SteamDB_Call_of_Duty_Clean.csv", parse_dates=["month"])
bf_data = pd.read_csv("../assets/clean/SteamDB_Battlefield_Clean.csv", parse_dates=["month"])

igdb_data = pd.read_csv("../assets/clean/IGDB_Clean.csv")
twitch_cs_data = pd.read_csv("../assets/clean/Twitch_Counter-Strike_Clean.csv", parse_dates=["month"])
youtube_cs_data = pd.read_csv("../assets/clean/Youtube_Counter-Strike_Clean.csv", parse_dates=["month"])

In [ ]:
merged_youtube_twitch = pd.concat([youtube_cs_data.set_index("month"), twitch_cs_data.set_index("month")], axis=1).reset_index()
merged_youtube_twitch = merged_youtube_twitch[merged_youtube_twitch['month'] >= pd.to_datetime("2016-11-01")]

merged_cs_youtube = pd.merge(cs_merged_data[['month','peak']], youtube_cs_data[['month','viewCount']], on="month", how="inner")
merged_cs_youtube = merged_cs_youtube.sort_values(by="viewCount", ascending=False)

merged_cs_twitch = pd.merge(cs_merged_data[['month','peak']], twitch_cs_data[['month','peak viewers']], on="month", how="inner")
merged_cs_twitch = merged_cs_twitch.sort_values(by="peak viewers", ascending=False)

## Visualizations

In [84]:
series = {
    # "All Counter-Strike (aggregate)": cs_merged_grouped,
    "Counter-Strike (1.6)": cs_data,
    "Counter-Strike: Source": css_data,
    "Counter-Strike: Condition Zero": cscz_data,
    "Counter-Strike: Global Offensive": csgo_data,
    "Counter-Strike 2": cs2_data,
}

colors = {
    # "All Counter-Strike (aggregate)":  "#0057B8",  # deep royal blue (highlighted)
    "Counter-Strike (1.6)":            "#009E73",  # green
    "Counter-Strike: Condition Zero":  "#7F7F7F",  # medium gray (subtle)
    "Counter-Strike: Source":          "#56B4E9",  # sky blue
    "Counter-Strike: Global Offensive":"#D55E00",  # vermillion (strong contrast vs blue)
    "Counter-Strike 2":                "#CC79A7",  # magenta
}

order = [
#     "All Counter-Strike (aggregate)",
    "Counter-Strike (1.6)",
    "Counter-Strike: Condition Zero",
    "Counter-Strike: Source",
    "Counter-Strike: Global Offensive",
    "Counter-Strike 2",
]

# Combine all series into one tidy DataFrame
frames = []
for name, df in series.items():
    df["game"] = name
    frames.append(df[["month", "peak", "game"]])

combined = pd.concat(frames, ignore_index=True)

# Build Altair chart
chart = (
    alt.Chart(combined)
    .mark_line()
    .encode(
        x=alt.X("month:T", title="Year"),
        y=alt.Y("peak:Q", title="Peak players"),
        color=alt.Color("game:N", title="Game/Version", scale=alt.Scale(domain=order, range=[colors[k] for k in order])),
        tooltip=["game", "month", "peak"],
    )
    .properties(width=950, height=450, title="Counter-Strike Playerbase Over Time")
)

chart

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [85]:
merged_yt_twitch = pd.concat([twitch_data_merged_grouped.set_index("month"), youtube_data_views_by_month.set_index("month")], axis=1).reset_index()
merged_yt_twitch.head()

,month,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched,viewCount
0,2005-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89144
1,2005-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19415
2,2005-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263357
3,2005-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1810148
4,2005-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10797827


In [86]:
# First line: views (left y-axis)
twitch_line = alt.Chart(merged_yt_twitch).mark_line(color="purple").encode(
    x=alt.X("month:T", title="Month"),
    y=alt.Y("Peak Viewers:Q", title="Twitch", axis=alt.Axis(titleColor="purple")),
    tooltip=["month", "Peak Viewers"]
)

# Second line: likes (right y-axis)
youtube_line = alt.Chart(merged_yt_twitch).mark_line(color="red").encode(
    x="month:T",
    y=alt.Y("viewCount:Q", title="Youtube", axis=alt.Axis(titleColor="red")),
    tooltip=["month", "viewCount"],
)

# Layer the two charts
chart = alt.layer(youtube_line, twitch_line).resolve_scale(
    y="independent"  # independent y-axes so they can differ
).properties(width=950, height=450, title="Counter-Strike Viewership Over Time")

chart

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [87]:
# Ensure proper dtypes
df1 = merged_yt_and_steamdb.copy()
df1["month"] = pd.to_datetime(df1["month"])
df1["year"] = df1["month"].dt.year

# Base scatter
scatter = (
    alt.Chart(df1)
    .mark_circle(size=60, opacity=0.6)
    .encode(
        x=alt.X("peak:Q", title="Playerbase (monthly peak, SteamDB)"),
        y=alt.Y("viewCount:Q", title="YouTube Viewership (monthly total)"),
        color=alt.Color("year:N", title="Year", legend=alt.Legend(columns=2)),
        tooltip=[
            alt.Tooltip("month:T", title="Month"),
            alt.Tooltip("peak:Q", title="Peak players", format=","),
            alt.Tooltip("viewCount:Q", title="Views", format=","),
        ]
    )
)

# Regression trendline (linear)
trend = (
    scatter
    .transform_regression("peak", "viewCount")
    .mark_line(size=3)
    .encode(color=alt.value("black"))
)

(scatter + trend).properties(
    title="Counter-Strike: Playerbase vs. YouTube Viewership (by month)"
)


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [88]:
merged_twitch_and_steamdb.head()

,month,peak,Peak Viewers
59,2021-10-01,888099.0,1950442.0
65,2022-04-01,1042536.0,1634605.0
87,2024-02-01,1431938.0,1553750.0
13,2017-12-01,638526.0,1320676.0
71,2022-10-01,1110627.0,1273072.0


In [89]:
# Ensure proper dtypes
df2 = merged_twitch_and_steamdb.copy()
df2["month"] = pd.to_datetime(df2["month"])
df2["year"] = df2["month"].dt.year

# Base scatter
scatter = (
    alt.Chart(df2)
    .mark_circle(size=60, opacity=0.6)
    .encode(
        x=alt.X("peak:Q", title="Playerbase (monthly peak, SteamDB)"),
        y=alt.Y("Peak Viewers:Q", title="Twitch Peak Viewership (monthly total)"),
        color=alt.Color("year:N", title="Year", legend=alt.Legend(columns=2)),
        tooltip=[
            alt.Tooltip("month:T", title="Month"),
            alt.Tooltip("peak:Q", title="Peak players", format=","),
            alt.Tooltip("Peak Viewers:Q", title="Peak Viewers", format=","),
        ]
    )
)

# Regression trendline (linear)
trend = (
    scatter
    .transform_regression("peak", "Peak Viewers")
    .mark_line(size=3)
    .encode(color=alt.value("black"))
)

(scatter + trend).properties(
    title="Counter-Strike: Playerbase vs. Twitch Peak Viewership (by month)"
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


### FPS Competitors

In [90]:
# load the heatmap matrix from assets
pair_heat = pd.read_csv("../assets/trends_heatmap_pairwise.csv", index_col=0)

In [91]:
# --- tidy + DATE CLEANING ---
long = (
    pair_heat.reset_index()
             .rename(columns={"index": "peak_row"})
             .melt(id_vars="peak_row", var_name="game", value_name="interest")
)

# Extract "peak game" and "(YYYY-MM)" then parse to datetime
long["peak_game"] = long["peak_row"].str.extract(r"peak@(.+?) \(")
long["peak_month_str"] = long["peak_row"].str.extract(r"\((\d{4}-\d{2})\)")
long["peak_month"] = pd.to_datetime(long["peak_month_str"], format="%Y-%m", errors="coerce")

# Nicer y labels: "Game (Mon YYYY)"
long["peak_label"] = np.where(
    long["peak_month"].notna(),
    long["peak_game"] + " (" + long["peak_month"].dt.strftime("%b %Y") + ")",
    long["peak_row"],
)

# --- ordering ---
# Rows: chronological by peak month (change to .sort_values("interest", ascending=False) for strongest-first)
row_order = (
    long.loc[long["game"] == long["peak_game"], ["peak_label", "peak_month"]]
        .drop_duplicates(subset=["peak_label"])  # guard against dup joins
        .sort_values("peak_month")["peak_label"]
        .tolist()
)

# Columns: by max interest across rows (helps readability)
col_order = (
    long.groupby("game")["interest"]
        .max()
        .sort_values(ascending=False)
        .index
        .tolist()
)


In [92]:
# --- color scaling ---
vmin = float(np.nanmin(long["interest"].values)) if not long["interest"].dropna().empty else 0.0
vmax = float(np.nanmax(long["interest"].values)) if not long["interest"].dropna().empty else 100.0
# In pairwise scaling, diagonal is 100 by construction; off-diagonals can exceed 100 if j > i in i's peak month.
# Use a dynamic upper bound so we don't clip >100 values.
color_scale = alt.Scale(domain=[max(0, vmin), vmax], scheme="blues")

# --- heatmap ---
heatmap = (
    alt.Chart(long)
    .mark_rect()
    .encode(
        y=alt.Y("peak_label:N", title="Peak month per game", sort=row_order),
        x=alt.X("game:N", title="Game", sort=col_order),
        color=alt.Color("interest:Q", title="Pairwise-scaled interest", scale=color_scale),
        tooltip=[
            alt.Tooltip("peak_game:N", title="Row peak of"),
            alt.Tooltip("peak_month:T", title="Peak month"),
            alt.Tooltip("game:N", title="Column game"),
            alt.Tooltip("interest:Q", title="Interest", format=".0f"),
        ],
    )
    .properties(
        title="Google Trends — Pairwise-at-peak Heatmap (2-term requests; row-diagonal = 100)",
        width={"step": 30},
        height={"step": 30},
    )
)

# Optional numeric labels (auto-contrasting color)
labels = (
    alt.Chart(long)
    .mark_text(size=9)
    .encode(
        y=alt.Y("peak_label:N", sort=row_order),
        x=alt.X("game:N", sort=col_order),
        text=alt.Text("interest:Q", format=".0f"),
        color=alt.condition(
            alt.datum.interest >= (0.6 * vmax), alt.value("white"), alt.value("black")
        ),
    )
)

(heatmap + labels).configure_axis(labelLimit=280).configure_view(stroke=None)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


### Heatmap Visualizations

In [93]:
pb_cs_data = cs_merged_grouped[cs_merged_grouped['month'] >= '2020-04-01']
pb_cod_data = pd.read_csv("../assets/SteamDB_Call_of_Duty_Clean.csv")
pb_val_data = pd.read_csv("../assets/GG_Valorant_Clean.csv")
pb_siege_data = pd.read_csv("../assets/GG_Rainbow_Six_Siege_Clean.csv")
pb_fn_data = pd.read_csv("../assets/GG_Fortnite_Clean.csv")
# drop the last row
pb_fn_data = pb_fn_data.iloc[:-1].copy()

pb_bf_data = pd.read_csv("../assets/SteamDB_Battlefield_Clean.csv")

# pb_cs_data.head()
# cod_data.head()
# val_data.head()
# siege_data.head()
pb_fn_data.tail()
# bf_data.head()

,Unnamed: 0,month,peak,gain,% gain
24,6,2025-04-01,2132493.0,-624636.0,-22.7
25,5,2025-05-01,2356174.0,223681.0,10.5
26,4,2025-06-01,5948454.0,3592280.0,152.5
27,3,2025-07-01,2396423.0,-3552031.0,-59.7
28,2,2025-08-01,3984350.0,1587927.0,66.3


In [94]:
def prep(df, name):
    out = df[['month','peak', '% gain']].copy()
    out['month'] = pd.to_datetime(out['month']) # Not needed
    return out.rename(columns={'peak': name})

In [95]:
cs   = prep(pb_cs_data, 'cs')
cod  = prep(pb_cod_data, 'cod')
val  = prep(pb_val_data, 'valorant')
sie  = prep(pb_siege_data, 'siege')
fn   = prep(pb_fn_data, 'fortnite')
bf   = prep(pb_bf_data, 'battlefield')

dfs = [cs, cod, val, sie, fn, bf]
merged = reduce(lambda l, r: pd.merge(l, r, on='month', how='outer'), dfs).sort_values('month')

# (optional) limit to 2020+ across all games to match CS slice
merged = merged[merged['month'] >= pd.Timestamp('2020-06-01')]


In [97]:
long = merged.melt(id_vars='month', var_name='game', value_name='players')

line = alt.Chart(long).mark_line().encode(
    x=alt.X('month:T', title='Month'),
    y=alt.Y('players:Q', title='Peak players'),
    color=alt.Color('game:N', title='Game'),
    tooltip=['month:T','game:N','players:Q']
).properties(width=900, height=350, title='Playerbase trends since 2020')

line


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [98]:
norm = long.groupby('game', group_keys=False).apply(
    lambda d: d.assign(players_z=(d['players']-d['players'].mean())/d['players'].std())
)

line_norm = alt.Chart(norm).mark_line().encode(
    x='month:T', y=alt.Y('players_z:Q', title='Z-score'),
    color='game:N', tooltip=['month:T','game:N','players_z:Q']
).properties(width=900, height=350, title='Normalized (z-score) playerbase trends')

line_norm


/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_73125/2812539738.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  norm = long.groupby('game', group_keys=False).apply(


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [99]:
# compute Pearson correlations on the wide numeric block
corr = merged.drop(columns=['month']).corr()

# tidy for Altair
corr_tidy = corr.stack().rename('corr').reset_index().rename(
    columns={'level_0':'game_x','level_1':'game_y'}
)

heat = alt.Chart(corr_tidy).mark_rect().encode(
    x=alt.X('game_x:N', sort=corr.columns.tolist(), title=''),
    y=alt.Y('game_y:N', sort=corr.columns.tolist(), title=''),
    color=alt.Color('corr:Q', scale=alt.Scale(scheme='redblue', domain=[-1,1]), title='Pearson r'),
    tooltip=['game_x:N','game_y:N', alt.Tooltip('corr:Q', format='.2f')]
).properties(width=400, height=400, title='Correlation of monthly peak players')

text = alt.Chart(corr_tidy).mark_text(baseline='middle').encode(
    x='game_x:N', y='game_y:N',
    text=alt.Text('corr:Q', format='.2f'),
    color=alt.condition('datum.corr > 0.5 || datum.corr < -0.5',
                        alt.value('black'),
                        alt.value('gray'))
)

heat + text


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [100]:
cs_only = corr_tidy.query("game_x == 'cs' & game_y != 'cs'").copy()
cs_only['game_y'] = cs_only['game_y'].str.title()  # cosmetic

cs_heat = alt.Chart(cs_only).mark_rect().encode(
    x=alt.X('game_y:N', title='Game'),
    y=alt.Y('game_x:N', title='', axis=alt.Axis(labels=False, ticks=False)),
    color=alt.Color('corr:Q', scale=alt.Scale(scheme='redblue', domain=[-1,1])),
    tooltip=['game_y:N', alt.Tooltip('corr:Q', format='.2f')]
).properties(width=400, height=60, title='Correlation with CS')

cs_text = alt.Chart(cs_only).mark_text(baseline='middle').encode(
    x='game_y:N', y='game_x:N', text=alt.Text('corr:Q', format='.2f')
)

cs_heat + cs_text


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [101]:
def sel_and_fix(df, name):
    out = df[['month', 'peak']].copy()

    # robust datetime coercion (handles 'YYYY-MM', 'YYYY/MM/DD', ints, etc.)
    out['month'] = pd.to_datetime(out['month'], errors='coerce', infer_datetime_format=True)

    # drop any bad rows that couldn't be parsed
    out = out.dropna(subset=['month']).sort_values('month')

    # remove timezone if present
    try:
        out['month'] = out['month'].dt.tz_localize(None)
    except (TypeError, AttributeError):
        pass

    return out.rename(columns={'peak': name})

# Fix all six
cs  = sel_and_fix(pb_cs_data,  'cs')
cod = sel_and_fix(pb_cod_data, 'cod')
val = sel_and_fix(pb_val_data, 'valorant')
sie = sel_and_fix(pb_siege_data,'siege')
fn  = sel_and_fix(pb_fn_data,  'fortnite')
bf  = sel_and_fix(pb_bf_data,  'battlefield')

# quick sanity check (all should show datetime64[ns])
print(cs.dtypes, cod.dtypes, val.dtypes, sie.dtypes, fn.dtypes, bf.dtypes)

# now this merge will succeed
merged = reduce(lambda l, r: pd.merge(l, r, on='month', how='outer'),
                [cs, cod, val, sie, fn, bf]).sort_values('month')

month    datetime64[ns]
cs              float64
dtype: object month    datetime64[ns]
cod             float64
dtype: object month       datetime64[ns]
valorant           float64
dtype: object month    datetime64[ns]
siege           float64
dtype: object month       datetime64[ns]
fortnite           float64
dtype: object month          datetime64[ns]
battlefield           float64
dtype: object


/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_73125/3455772743.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  out['month'] = pd.to_datetime(out['month'], errors='coerce', infer_datetime_format=True)
/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_73125/3455772743.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  out['month'] = pd.to_datetime(out['month'], errors='coerce', infer_datetime_format=True)
/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_73125/3455772743.py:5: UserWarning: The argument 'infer_datetime_form

In [102]:
heat = alt.Chart(long_z).mark_rect().encode(
    # yearmonth() makes x discrete → one tile per month
    x=alt.X('yearmonth(month):T', title='Month',
            axis=alt.Axis(format='%Y %b')),
    y=alt.Y('game:N', title='Game', sort=['battlefield','cod','fortnite','siege','valorant','cs']),
    color=alt.Color('z_score:Q',
                    scale=alt.Scale(scheme='redblue', domain=[-2, 2]),
                    title='Z-score'),
    tooltip=[
        alt.Tooltip('yearmonth(month):T', title='Month', format='%Y %b'),
        alt.Tooltip('game:N', title='Game'),
        alt.Tooltip('z_score:Q', title='Z', format='.2f')
    ]
).properties(width=900, height=240, title='Z-scored Playerbases Over Time')


NameError: name 'long_z' is not defined

In [ ]:
heat = heat.transform_timeunit(ym='yearmonth(month)').transform_impute(
    impute='z_score', key='ym', groupby=['game'], value=None
).encode(x='ym:T')

heat

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
